In [ ]:
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for the model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# for feature engineering
from sklearn.preprocessing import StandardScaler
from feature_engine import missing_data_imputers as mdi
from feature_engine import discretisers as dsc
from feature_engine import categorical_encoders as ce
import seaborn as sns
from feature_engine import variable_transformers as vt
import scipy.stats as stats
pd.pandas.set_option('display.max_columns', None)
from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from feature_engine.outlier_removers import Winsorizer
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
##LOAD DATASET

In [ ]:
data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
datas = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
data.head()

In [ ]:
# let's inspect the type of variables in pandas

In [ ]:
for var in data.columns:
    print(var,data[var].dtypes)

In [ ]:
#find categorical variables

In [ ]:
categorical = [var for var in data.columns if data[var].dtype=='O']

print('There are {} categorical variables'.format(len(categorical)))

In [ ]:
data[categorical].head()

In [ ]:
# make a list of the numerical variables 

In [ ]:
numerical = [var for var in data.columns if data[var].dtype!='O']

In [ ]:
data[numerical].head()

In [ ]:
# list of variables that contain year information


In [ ]:
year_vars = [var for var in numerical if 'Yr' in var or 'Year' in var]

year_vars

In [ ]:
data[year_vars].head()

In [ ]:
# plot median house price per month in which it was sold

In [ ]:
data.groupby('MoSold')['SalePrice'].median().plot()
plt.title('House price variation along the year')
plt.ylabel('median House price')

In [ ]:
# let's visualise the values of the discrete variables

In [ ]:
discrete = []

for var in numerical:
    if len(data[var].unique()) < 20 and var not in year_vars:
        print(var, ' values: ', data[var].unique())
        discrete.append(var)
print()
print('There are {} discrete variables'.format(len(discrete)))

In [ ]:
data[discrete].head()

In [ ]:
# find continuous variables

In [ ]:
numerical = [var for var in numerical if var not in discrete and var not in [
    'Id', 'SalePrice'] and var not in year_vars]

print('There are {} numerical and continuous variables'.format(len(numerical)))

In [ ]:
data[numerical].head()

In [ ]:
#imputing missing variables for categorical variables

In [ ]:
data[categorical].isnull().sum()

In [ ]:
#impute Missing values with an indicator "M"

In [ ]:
for var in categorical:
    data[var] = np.where(data[var].isnull(),'M',data[var])
    datas[var] = np.where(datas[var].isnull(),'M',datas[var])

In [ ]:
data[categorical].isnull().sum()

In [ ]:
#Missing values replaced.

In [ ]:
#now lets check for missing values in discrete variables


In [ ]:
data[discrete].isnull().sum()

In [ ]:

####NOTE:::replace here
for var in discrete:
    datas[var] = np.where(datas[var].isnull(),datas[var].mode(),datas[var])
datas[discrete].isnull().sum()

In [ ]:
#lets check for missing values in numerical varibels now

In [ ]:
data[numerical].isnull().sum()

In [ ]:
'''LotFrontage: Linear feet of street connected to property , 
thus we can assume there are missing values as these houses have no conectivity to street area
we will impute these missing values with arbitary value -1 to mark this observation'''

In [ ]:
data['LotFrontage'] = np.where(data['LotFrontage'].isnull(),-1,data['LotFrontage'])
datas['LotFrontage'] = np.where(datas['LotFrontage'].isnull(),-1,datas['LotFrontage'])

In [ ]:
data['MasVnrArea'] = np.where(data['MasVnrArea'].isnull(),data['MasVnrArea'].mean(),data['MasVnrArea'])
datas['MasVnrArea'] = np.where(datas['MasVnrArea'].isnull(),datas['MasVnrArea'].mean(),datas['MasVnrArea'])
datas['BsmtFinSF1'] = np.where(datas['BsmtFinSF1'].isnull(),datas['BsmtFinSF1'].mean(),datas['BsmtFinSF1'])
datas['BsmtFinSF2'] = np.where(datas['BsmtFinSF2'].isnull(),datas['BsmtFinSF2'].mean(),datas['BsmtFinSF2'])
datas['BsmtUnfSF'] = np.where(datas['BsmtUnfSF'].isnull(),datas['BsmtUnfSF'].mean(),datas['BsmtUnfSF'])
datas['TotalBsmtSF'] = np.where(datas['TotalBsmtSF'].isnull(),datas['TotalBsmtSF'].mean(),datas['TotalBsmtSF'])
datas['GarageArea'] = np.where(datas['GarageArea'].isnull(),datas['GarageArea'].mean(),datas['GarageArea'])

In [ ]:
data[numerical].isnull().sum()

In [ ]:
#Missing values for numerical features imputed.

In [ ]:
datas[numerical].isnull().sum()
####NOTE:::do mean median imputing for dataset...

In [ ]:
#one hot categorical encoding for categorical variables.

In [ ]:
y_df = data['SalePrice']

In [ ]:
#re-casting discrete variables to categorical

In [ ]:
data[discrete] =data[discrete].astype('O')
datas[discrete] =datas[discrete].astype('O')


In [ ]:
#rare-label encoding
del data['SalePrice']

In [ ]:
rareenc = ce.RareLabelCategoricalEncoder(
        tol=0.05, n_categories=6, variables=categorical+discrete)
rareenc.fit(data)
data = rareenc.transform(data)
datas = rareenc.transform(datas)
##space for datas

In [ ]:
enc = OneHotCategoricalEncoder(
    top_categories=None,
    variables=categorical, # we can select which variables to encode
    drop_last=True) 
enc.fit(data,y_df)
data= enc.transform(data)
datas = enc.transform(datas)

In [ ]:
datas

In [ ]:
#ordinal encoding for discrete variables

In [ ]:
orden = ce.OrdinalCategoricalEncoder(
        encoding_method='ordered', variables=discrete)
orden.fit(data,y_df)
data = orden.transform(data)
datas = orden.transform(datas)
##space for datas

In [ ]:
datas[discrete].isnull().sum()
datas['FullBath'] = np.where(datas['FullBath'].isnull(),datas['FullBath'].mode(),datas['FullBath'])
datas['Fireplaces'] = np.where(datas['Fireplaces'].isnull(),datas['Fireplaces'].mode(),datas['Fireplaces'])
datas['GarageCars'] = np.where(datas['GarageCars'].isnull(),datas['GarageCars'].mode(),datas['GarageCars'])
datas[discrete].isnull().sum()

In [ ]:
#now we will assess normality for numerical continuous variables

In [ ]:
def diagnostic_plots(df, variable):
    
    # function to plot a histogram and a Q-Q plot
    # side by side, for a certain variable
    
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    df[variable].hist()

    plt.subplot(1, 2, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)

    plt.show()


In [ ]:
for var in numerical:
    print(var)
    diagnostic_plots(data, var)

In [ ]:
#transforming variables to see best function

In [ ]:
from feature_engine import variable_transformers as vt

In [ ]:
transform = ['LotArea','BsmtUnfSF','GrLivArea','GarageArea']
lt = vt.YeoJohnsonTransformer(variables = transform)
lt.fit(data)
data = lt.transform(data)
datas = lt.transform(datas)

In [ ]:
#I selected variables for transformation by selecting individual graphs

In [ ]:
#Outlier enginerring
windsoriser = Winsorizer(distribution='quantiles', # choose from skewed, gaussian or quantiles
                          tail='both', # cap left, right or both tails 
                          fold=0.05,
                          variables=numerical)
windsoriser.fit(data)
data_tf = windsoriser.transform(data)
datas = windsoriser.transform(datas)

In [ ]:
data = data_tf

In [ ]:
#engineering temporal variables

In [ ]:
def elapsed_years(df, var):
    # capture difference between year variable and
    # year the house was sold
    
    df[var] = df['YrSold'] - df[var]
    return df

In [ ]:
for var in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    data = elapsed_years(data, var)
    datas = elapsed_years(datas, var)

In [ ]:
data[['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']].head()

In [ ]:
# drop YrSold
data.drop('YrSold', axis=1, inplace=True)
datas.drop('YrSold', axis=1, inplace=True)

In [ ]:
del data['Id']

In [ ]:
del datas['Id']

In [ ]:
#feature Selection

In [ ]:
#removing constant features
from sklearn.feature_selection import VarianceThreshold

In [ ]:
sel = VarianceThreshold(threshold=0)
sel.fit(data)  # fit finds the features with zero variance


In [ ]:
sum(sel.get_support())

In [ ]:
data= sel.transform(data)
datas = sel.transform(datas)

data.shape , datas.shape

In [ ]:
datas

In [ ]:
data = pd.DataFrame(data)
datas = pd.DataFrame(datas)

In [ ]:
#removing co-related features
data

In [ ]:
#del data[32]

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(data, 0.8)
len(set(corr_features))

In [ ]:
data.drop(labels=corr_features, axis=1, inplace=True)
datas.drop(labels=corr_features, axis=1, inplace=True)

data.shape #, X_test.shape

In [ ]:
#datas.drop(labels=corr_features, axis=1, inplace=True)
datas.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    y_df,
                                                    test_size=0.1,
                                                    random_state=0)

X_train.shape, X_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(X_train)
x_test = sc.transform(X_test)
x_test2 = sc.transform(datas)

In [ ]:
#Modelling

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
model_xgb=xgb.XGBRegressor(colsample_bytree=0.7, gamma=0.04, 
                             learning_rate=0.05, max_depth=5, 
                             min_child_weight=6, n_estimators=2000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.7, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
model_xgb.fit(x, y_train)
xgb_train_pred = model_xgb.predict(x)
xgb_pred = np.expm1(model_xgb.predict(x_test))
print(rmsle(y_train, xgb_train_pred))


In [ ]:
xgb_pred2 = model_xgb.predict(x_test2)

In [ ]:
mean_squared_error(y_train, xgb_train_pred)